## Test code for style transfer with food project

In [ ]:
# Import package
import os
from data import CreateDataLoader
from cycle_gan_model import CycleGANModel
from util.visualizer import save_images
from util import html
import config

In [ ]:
# Configuration
config.ntest = 2000
config.results_dir = '../test_results/'
config.aspect_ratio = 1.0
config.eval = True
config.num_test = 50
config.phase = 'test'
config.model = 'test'
config.loadSize = config.fineSize
config.isTrain = False

config.num_threads = 1   
config.batch_size = 1    
config.serial_batches = True  
config.no_flip = True    
config.display_id = -1   # no visdom display

model = CycleGANModel()
model.initialize(config)
model.setup(config)
print("Network Model Created")
data_loader = CreateDataLoader(config)
dataset = data_loader.load_data()

In [ ]:
# Testing
# create a container for results
web_dir = os.path.join(config.results_dir, config.name, '%s_%s' % (config.phase, config.epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (config.name, config.phase, config.epoch))
    
for i, data in enumerate(dataset):
    if i >= config.num_test:
        break
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    img_path = model.get_image_paths()
    if i % 5 == 0:
        print('processing (%04d)-th image... %s' % (i, img_path))
    save_images(webpage, visuals, img_path, aspect_ratio=config.aspect_ratio, width=config.display_winsize)

# Save results
webpage.save()